In [1]:
import pandas as pd
import collections as c
import numpy as np
import heapq

In [24]:
class RuleData:
    def __init__(self,rule=((),()) ,support=0.0, confidence=0.0, expFlag=True):
        self.rule=rule
        self.support=support
        self.confidence=confidence
        self.expFlag=expFlag

In [3]:
#data = pd.read_csv(r'Datasets/chess.txt',sep=" ",header=None)
data = pd.read_csv(r'Datasets/sample.txt',sep=" ",header=None)
data.head()
#df = pd.read_csv(filepath, chunksize=1, header=None, encoding='utf-8')

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN


In [4]:
# data.drop(37,axis=1,inplace=True)
# data.head()
data

,0,1,2,3,4
0,1,2,3,4.0,5.0
1,1,2,4,5.0,NaN
2,2,3,5,NaN,NaN
3,1,2,4,5.0,NaN
4,1,2,3,5.0,NaN
5,2,3,4,NaN,NaN


In [5]:
itemset = list(set(data.values.flatten()))
itemset

[nan, 1.0, 2.0, 3.0, 4.0, 5.0, nan, nan, nan, nan, nan, nan]

In [6]:
tidsets={}
for i in range(len(data)):
    for j in range(len(data.columns)):
        val = data.iloc[i][j]
        if pd.notna(val):
            if val in tidsets:
                tidsets[val].append(i)
            else:
                tidsets[val]=[i]
tidsets

{1.0: [0, 1, 3, 4],
 2.0: [0, 1, 2, 3, 4, 5],
 3.0: [0, 2, 4, 5],
 4.0: [0, 1, 3, 5],
 5.0: [0, 1, 2, 3, 4]}

In [7]:
# tidsets=[[]]
# for i in itemset:
#     tidsets.append([])
# for i in range(len(data)):
#     for j in range(len(data.columns)):
#         val = data.iloc[i][j]
#         tidsets[val-1].append(i)
# tidsets

In [8]:
#[j for j in range(len(data)) if max((data==itemset[0]).iloc[j])==True]
data.iloc[0].values

array([1., 2., 3., 4., 5.])

In [9]:
minsup=0
minconf=0.8
k=1000
l=list()
r=list()

In [10]:
# itemcount = dict(c.Counter(data.values.flatten()))
# itemcount

In [22]:
class MyHeap(object):
    def __init__(self, initial=None, key=lambda x:x):
        self.key = key
        if initial:
                self._data = [(self.key(item), item) for item in initial]
                heapq.heapify(self._data)
        else:
                self._data = []

    def push(self, item):
        heapq.heappush(self._data, (self.key(item), item))

    def pop(self):
        return heapq.heappop(self._data)[1]
    def length(self):
        return len(self._data)

In [12]:
def support(rule=((),())):
    if(isinstance(rule[0],int) or isinstance(rule[0],float)):
        if(isinstance(rule[1],int) or isinstance(rule[1],float)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int) or isinstance(rule[1],float)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    items=rule[0]+rule[1]
    sumRule=0
    for i in range(len(data)):
        flagRule=True
        for j in items:
            if j not in list(data.iloc[i,:]):
                flagRule=False
                break
        if(flagRule):
            sumRule+=1
    return sumRule/len(data)

In [13]:
support(((2,3),(1)))

0.3333333333333333

In [14]:
def confidence(rule=((),())):
    print(rule)
    if(isinstance(rule[0],int) or isinstance(rule[0],float)):
        if(isinstance(rule[1],int) or isinstance(rule[1],float)):
            rule=[[rule[0]],[rule[1]]]
        elif(isinstance(rule[1],tuple)):
            rule=[[rule[0]],list(rule[1])]
    elif(isinstance(rule[1],int) or isinstance(rule[1],float)):
        if(isinstance(rule[0],tuple)):
            rule=[list(rule[0]), [rule[1]]]
    items=rule[0]+rule[1]
    print(rule)
    print(items)
    sumRule=0
    sumAntecedant=0
    #The following can be optimized by bringing both under single loop block
    for i in range(len(data)):
        flagRule=True
        for j in items:
            if j not in list(data.iloc[i,:]):
                flagRule=False
                break
        if(flagRule):
            sumRule+=1
            print(sumRule)
    for i in range(len(data)):
        flagAntecedant=True
        for j in rule[0]:
            if j not in list(data.iloc[i,:]):
                flagAntecedant=False
                break
        if(flagAntecedant):
            sumAntecedant+=1
            print(sumAntecedant)
    return sumRule/sumAntecedant

In [15]:
ruleDetails=dict()
L=MyHeap(key=lambda x:-x.support)

In [16]:
confidence(((2,3),(1)))

((2, 3), 1)
[[2, 3], [1]]
[2, 3, 1]
1
2
1
2
3
4


0.5

In [20]:
def save(L,k,rule=((),())):
    L.push(RuleData(rule,support(rule),confidence(rule)))
    ruleData=RuleData()
    q=list()
    for i in range(min(len(L),k)):
        q+=[L.pop()]
    ruleData=L.pop()
    while(ruleData.support==q[-1].support):
        q+=[ruleData]
    L=MyHeap(q, key=lambda x:-x.support)
    L
            
            

In [23]:
items = list(tidsets.keys())
for i in range(len(items)):
    for j in range(i+1,len(items)):
        if itemcount[items[i]]>minsup and itemcount[items[j]]>minsup:
            conf1=confidence(((items[i]),(items[j])))
            conf2=confidence(((items[j]),(items[i])))
            if conf1>minconf:
                ruleDetails[(items[i],items[j])]=support((items[i],items[j]))
                save(L,k,(items[i],items[j]))
            if conf2>minconf:
                ruleDetails[(items[j],items[i])]=support((items[j],items[i]))
                save(L,k,(items[j],items[i]))

(1.0, 2.0)
[[1.0], [2.0]]
[1.0, 2.0]
1
2
3
4
1
2
3
4
(2.0, 1.0)
[[2.0], [1.0]]
[2.0, 1.0]
1
2
3
4
1
2
3
4
5
6
(1.0, 2.0)
[[1.0], [2.0]]
[1.0, 2.0]
1
2
3
4
1
2
3
4


TypeError: '<' not supported between instances of 'RuleData' and 'RuleData'

In [ ]:
# summ=0
# for i in range(data.shape[0]):
#     flag=True
#     for j in ((2,3)):
#         if j not in list(data.iloc[i,:]):
#             flag=False
#             break
#     if(flag):
#         print(i)
#         summ+=1